<a href="https://colab.research.google.com/github/samitha278/multilayer-perceptron-char-lm/blob/main/mlp_char_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving names.txt to names.txt


In [ ]:
words = open("names.txt","r").read().splitlines()

In [ ]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

### Building dataset

In [15]:
block_size = 3


xs = []
ys = []


context = [0]*block_size



for w in words:

  for ch in w + '.':

    xs.append(context)
    next = stoi[ch]
    ys.append(next)


    context = context[1:] + [next]


x = torch.tensor(xs)
y = torch.tensor(ys)

In [17]:
x.shape,y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

### Embedding

In [21]:
C = torch.randn((27,2))
C

tensor([[ 0.2781,  0.4918],
        [ 1.3179,  1.4982],
        [ 0.6720, -0.9691],
        [-2.1497, -2.1738],
        [ 0.7141, -0.8387],
        [-0.1015,  1.2391],
        [-0.1621,  0.8586],
        [-1.9788,  0.5472],
        [-1.1494, -0.5262],
        [-0.3381, -0.3613],
        [-1.3245,  2.0941],
        [ 1.0029,  1.7922],
        [-0.8868, -0.0446],
        [-0.2211, -0.7242],
        [ 1.4162,  1.4105],
        [ 1.1210,  0.7991],
        [ 0.4431,  1.8579],
        [ 0.0807, -0.7559],
        [-1.5218, -0.7468],
        [ 2.1135, -1.6136],
        [-2.5826, -0.6771],
        [ 0.0170, -0.6379],
        [-0.6740,  0.7792],
        [ 0.0028, -0.2878],
        [-0.7103,  0.1821],
        [-0.6147,  0.0587],
        [ 1.1983, -0.6559]])

In [22]:
C[5]

tensor([-0.1015,  1.2391])

In [23]:
C[torch.tensor([2,3,5,5,5])]

tensor([[ 0.6720, -0.9691],
        [-2.1497, -2.1738],
        [-0.1015,  1.2391],
        [-0.1015,  1.2391],
        [-0.1015,  1.2391]])

In [25]:
C[torch.tensor([[2,3],[3,2],[5,5],[2,5]])]

tensor([[[ 0.6720, -0.9691],
         [-2.1497, -2.1738]],

        [[-2.1497, -2.1738],
         [ 0.6720, -0.9691]],

        [[-0.1015,  1.2391],
         [-0.1015,  1.2391]],

        [[ 0.6720, -0.9691],
         [-0.1015,  1.2391]]])

In [26]:
emb = C[x]
emb.shape

torch.Size([228146, 3, 2])

In [33]:
x[1],C[[0,5]],emb[1]

(tensor([0, 0, 5]),
 tensor([[ 0.2781,  0.4918],
         [-0.1015,  1.2391]]),
 tensor([[ 0.2781,  0.4918],
         [ 0.2781,  0.4918],
         [-0.1015,  1.2391]]))

### Implementing hidden layer